In [16]:
import torch

In [17]:
model = torch.load("../Models/lif_snn_mnist.ckpt")
model.eval()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=1024, out_features=64, bias=False)
  (2): LIFNode(
    v_threshold=1.0, v_reset=0.0, detach_reset=False, tau=2.0
    (surrogate_function): Sigmoid(alpha=1.0, spiking=True)
  )
  (3): Linear(in_features=64, out_features=10, bias=False)
  (4): LIFNode(
    v_threshold=1.0, v_reset=0.0, detach_reset=False, tau=2.0
    (surrogate_function): Sigmoid(alpha=1.0, spiking=True)
  )
)

In [6]:
w_12 = model[1].weight.detach().cpu().numpy()

In [18]:
w_12 = model[1].weight.detach().cpu().numpy()
w_23 = model[3].weight.detach().cpu().numpy()

In [ ]:
w_12 = model[1].weight.detach().cpu().numpy()
w_23 = model[3].weight.detach().cpu().numpy()
w_34 = model[5].weight.detach().cpu().numpy()

In [7]:
# print(w_12.shape, w_23.shape, w_34.shape)
# print(w_12.shape, w_23.shape)
print(w_12.shape)

(10, 1024)


In [ ]:
f = open('../Models/MNIST_largescale_4/weights.txt', 'w')

tmp = 1024
tmp_1 = 0
for i in range(1024):
    print(i + tmp_1, file = f, end = " ")
    for j in range(256):
        print(j + tmp, file = f, end = " ");
        print(w_12[j][i], file = f, end = " ");
    print("", file = f)
        
tmp = 1024 + 256
tmp_1 = 1024
for i in range(256):
    print(i + tmp_1, file = f, end = " ")
    for j in range(64):
        print(j + tmp, file = f, end = " ");
        print(w_23[j][i], file = f, end = " ");
    print("", file = f)
    
tmp = 1024 + 256 + 64
tmp_1 = 1024 + 256
for i in range(64):
    print(i + tmp_1, file = f, end = " ")
    for j in range(10):
        print(j + tmp, file = f, end = " ");
        print(w_34[j][i], file = f, end = " ");
    print("", file = f)
f.close()

In [11]:
f = open('../Models/MNIST_largescale_3_2/weights.txt', 'w')

tmp = 1024
tmp_1 = 0
for i in range(1024):
    print(i + tmp_1, file = f, end = " ")
    for j in range(256):
        print(j + tmp, file = f, end = " ");
        print(w_12[j][i], file = f, end = " ");
    print("", file = f)
        
tmp = 1024 + 256
tmp_1 = 1024
for i in range(256):
    print(i + tmp_1, file = f, end = " ")
    for j in range(10):
        print(j + tmp, file = f, end = " ");
        print(w_23[j][i], file = f, end = " ");
    print("", file = f)
for i in range(10):
    print(tmp + i, file = f)

f.close()

In [19]:
f = open('../Models/MNIST_largescale_3/weights.txt', 'w')

tmp = 1024
tmp_1 = 0
for i in range(1024):
    print(i + tmp_1, file = f, end = " ")
    for j in range(64):
        print(j + tmp, file = f, end = " ");
        print(w_12[j][i], file = f, end = " ");
    print("", file = f)
        
tmp = 1024 + 64
tmp_1 = 1024
for i in range(64):
    print(i + tmp_1, file = f, end = " ")
    for j in range(10):
        print(j + tmp, file = f, end = " ");
        print(w_23[j][i], file = f, end = " ");
    print("", file = f)
for i in range(10):
    print(tmp + i, file = f)

f.close()

In [8]:
f = open('../Models/MNIST_largescale_2/weights.txt', 'w')

tmp = 1024
tmp_1 = 0
for i in range(1024):
    print(i + tmp_1, file = f, end = " ")
    for j in range(10):
        print(j + tmp, file = f, end = " ");
        print(w_12[j][i], file = f, end = " ");
    print("", file = f)
        
for i in range(10):
    print(tmp + i, file = f)

f.close()

In [ ]:
f = open('../Models/weights1.txt', 'w')
for i in range(256):
    for j in range(64):
        print(w_12[j][i], file = f, end = ' ')
    print("", file = f)
f.close()

In [ ]:
f = open('../Models/weights2.txt', 'w')
for i in range(64):
    for j in range(10):
        print(w_23[j][i], file = f, end = ' ')
    print("", file = f)
f.close()

In [ ]:
import torchvision

test_dataset = torchvision.datasets.MNIST(
        root="../Datasets/",
        train=False,
        download=True,
        transform=torchvision.transforms.Compose([
            torchvision.transforms.Resize(16),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(
                (0.1307,), (0.3081,))
        ])
    )

test_data_loader = torch.utils.data.DataLoader(
        dataset=test_dataset,
        batch_size=64,
        shuffle=False
)

In [ ]:
test_data_loader

In [ ]:
from spikingjelly.clock_driven import encoding
encoder = encoding.PoissonEncoder()
cnt = 0
f1 = open('../Datasets/MNIST_16/input.txt', 'w')
for img, label in test_data_loader:
    cnt += 1
    #if (cnt > 1):
    #    break;
    rg = 64
    if (cnt > 156):
        rg = 16
    for i in range(rg):
        for j in range(16):
            for k in range(16): 
                print(int(encoder(img[i][0][j][k]).float().tolist()), file = f1, end = ' ')
        print("", file = f1)
f1.close()
print(cnt * 64)

In [ ]:
f2 = open('../Datasets/MNIST_16/output.txt', 'w')
cnt = 0;
for img, label in test_data_loader:
    cnt += 1
    #if (cnt > 1):
    #    break;
    rg = 64
    if (cnt > 156):
        rg = 16
    for i in range(rg):
        a = int(label[i].tolist())
        print(a, file = f2)
f2.close()

In [ ]:
import math
device = torch.device('cpu')

for p in model[1].parameters():
    for v in p.data:
        print(v)

x = torch.Tensor([-1, -2, -3], device = device)
x.apply_(lambda x: x if x >= 0 else 0.01 * math.exp(x))
#for v in x:
#    v = -v
#    print(v)